### Cloud Instance Groups and Instance Templates<br>

***

<br>

Instance groups are a great way to set up a group of identical servers<br>
using conjunction of instance groups, templates handles the instance property to deploy instance groups to your environment<br>

Instance group is a collection of VM instances that you can manage as a single entity<br>
Compute engine offers 2 kinds of VM instance groups: Managed vs. Unmanaged<br>

##### **Managed Instance Groups(MIGs):**<br>

let you operate multiple identical VMs, you can make workload scalable and highly available<br>
by taking advantage of automated MIG services<br>
auto-scaling, auto-healing, regional and zonal deployments and automatic updating<br>

**Use cases:**<br>
- Stateless - serving workloads: website frontend, web servers, web apps<br>
- Stateless Batch - high performance or high throughput compute workloads<br>
- Stateful Workloads - use stateful managed instance groups<br>

- (High availability) **Autohealing**<br>
    - Keeps VMs in Running State<br>
    - Recreate VMs not in Running State<br>
    - Application-based autohealing<br>
    - Recreate VMs when app is frozen or has crashed<br>
    MIGs keep signal and recreate instances that are unhealthy<br>
- (High availability) **Regional (Multi-zone)**<br>
    - Zonal or regional MIGs<br>
    - Regional provides higher availability<br>
    - Zonal MIGs are in one zone only<br>
    - Google recommends regional MIGs<br>
- (High availability / Scalability) **Load Balancing**<br>
    - LB can use instance groups to serve traffics<br>
    - Work together to know how much traffic can be handled<br>
    - LB health checks do not send traffic to unhealthy instances<br>
- (Scalability) **Autoscaling**<br>
    - Dynamically add/remove instances from MIG<br>
    - Scale up to meet load demands<br>
    - Shrink as the load decreases to reduce costs<br>
- (Updates) **Auto-updating**<br>
    - deploy new versions of software to instances<br>
    - update deployment happens automatically<br>
    - perform rolling updates - allow updates with no downtime, by incrementally updating to the new one<br>
    - partial rollouts for canary testing - way to reduce risk and validate the software by releasing software to small % of users<br>
    also known as Staged Rollouts, it's the best practice in DevOps and software development<br>

- Reduce the cost of workload by using **Preemptible instances** in instance group<br>
- You can also deploy **containers** to the instances<br>
- **Network and Subnet** - when creating MIGs you must define VPC network that it'll reside in<br>
<br>

##### **Unmanaged Instance Groups**<br>

they also let you load balance across VMs, but you need to manage<br>

- Load Balancing<br>
UMIGs do not support autoscaling, autoupdating, autohealing, multizone, or use of instance templates<br>
not a good fit for deploying high available or scalable workload<br>

You should only use it if you need to apply to mixed types of VM instances<br>
or if you need to manage the instances yourself<br>
so UMIGs are designed for very special case where you'd need to mix instance types<br>
<br>

##### **Instance Template**<br>

instance template is a resource to create VM instances and manage instance groups<br>
Machine type, boot disk image, container image, labels -> Template<br>
Use to create MIG or VM -> Template<br>

If you want to create a group of identical instances, **you must use an insantce template to create a MIG**<br>
Existing instance template cannot be updated or changed - if you need to change, create a new instance template<br>
Save the config of an existing VM and create a new one<br>
> gcloud instance-templates create<br>

To make changes, you can create another one with similar properties using the console<br>
You can use custom or public images in your instance template<br>



**Demo**<br>

Load balancer + Instance group in practice<br>

- **Create an instance template**<br>

Check default VPC created<br>
Google Cloud -> Compute Engine -> Instance templates -> create instance template<br>
there is no cost for creating instance template, but it does show the cost of each instance based on the template you create<br>
name: template1<br>
machine config: N1, F1-micro<br>
firewall: allow HTTP traffic<br>
startup script: [paste in the script: 14:08:00]<br>
<br>

- **Use it to create instance group**<br>

Compute Engine -> Instance groups -> create an instance group<br>
Select 'new managed instance group (stateless)'<br>
name: group1<br>
location: multiple zone<br>
region: US-east1<br>
zone: b,c,d<br>
instance template: template1<br>
min # of instance: 3<br>
max # of instance: 6<br>
autohealing: create a new health check<br>

new health check<br>
name: healthy1<br>
leave all default and create it<br>
<br>

- **Allow health check in firewall**<br>

Google Cloud -> VPC Network -> Firewall -> Create firewall<br>
name: allow-health-check<br>
targets: all instances in the network<br>
source filter: IP ranges, 130.211.0.0/22, 35.191.0.0/16 - IP for Google's health check probe<br>
protocols and ports: TCP: 80<br>
<br>

- **Create a load balancer with a new backend**<br>

Google Cloud -> Network Services -> Load Balancing -> Create load balancer<br>
Pick HTTP(S) Load Balancer - because we're serving external traffic<br>
Internet facing or internal only: from internet to my VMs<br>
name: lb1<br>
click backend config: backend services: create a backend service<br>

new backend service<br>
name: backend1<br>
backend type: instance group<br>
instance group: group1<br>
port: 80<br>
health check: healthy1<br>

click frontend config:<br>
name: frontend-service1<br>
network service tier: premium<br>
IP ver: IPv4, IP address: Ephemeral<br>
port: 80<br>
<br>



Copy the IP address and paste it in new tab<br>
you'll see your deployed website<br>

- **Stress testing**<br>

Google Cloud -> Compute Engine -> VM Instance -> SSH into Instance1<br>
> stress --cpu 2 --io 1 --vm 1 --vm-bytes 128M --timeout 30s<br>

in a minute or two, you'll see some new instances created by the instace group<br>

- **Scale in**<br>

there is a 10 min restriction after scaling out, but there is a work around<br>
click rolling restart/replace -> click restart<br>
wait for few minutes<br>
